## Install the required libraries.
First, let’s install the necessary packages and connect your SERPAPI account using composio cli so agents can use it. Run the following commands on your terminal:

In [ ]:
!pip install composio-crewai
!pip install crewai
!pip install langchain-google-genai

In [ ]:
!composio login
!composio add serpapi

## Imports and Environment Setup
In your Python script, import the necessary libraries and set up your environment variables.

In [ ]:
from crewai import Agent, Task, Crew, Process
from composio_crewai import ComposioToolSet, Action, App
from langchain_google_genai import ChatGoogleGenerativeAI
import os

## Initialise tools and LLM
We use Composio to initialise our tools.


In [ ]:
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

# Initialize the language model
llm = ChatGoogleGenerativeAI(
model="gemini-pro", verbose=True, temperature=0.9, google_api_key=os.getenv("GOOGLE_API_KEY")
)

# Define tools for the agents
composio_toolset = ComposioToolSet()
tools = composio_toolset.get_actions(actions=[Action.SERPAPI_SEARCH])

## Defining the Agent

Define the roles and goals of our agents. There are four agents in use here: Investment Researcher, Investment Analyst, and Investment Recommender agents.

In [ ]:
 # Define the Investment Researcher agent
 researcher = Agent(
     role='Investment Researcher',
     goal='Use SERP to research the top 2 results based on the input given to you and provide a report',
     backstory="""
     You are an expert Investment researcher. Using the information given to you, conduct comprehensive research using
     various sources and provide a detailed report. Don't pass in location as an argument to the tool
     """,
     verbose=True,
     allow_delegation=True,
     tools=tools,
     llm=llm
 )
 # Define the Investment Analyst agent
 analyser = Agent(
     role='Investment Analyst',
     goal='Analyse the stock based on information available to it, use all the tools',
     backstory="""
     You are an expert Investment Analyst. You research on the given topic and analyse your research for insights.
     Note: Do not use SERP when you're writing the report
     """,
     verbose=True,
     tools=tools,
     llm=llm
 )

 # Define the Investment Recommender agent
 recommend = Agent(
     role='Investment Recommendation',
     goal='Based on the analyst insights, you offer recommendations',
     backstory="""
     You are an expert Investment Recommender. You understand the analyst insights and with your expertise suggest and offer
     advice on whether to invest or not. List the Pros and Cons as bullet points
     """,
     verbose=True,
     tools=tools,
     llm=llm
 )



## Defining the Task and Kickoff the Process

Create a task for the analyst agent and execute the process.

In [ ]:
# Get user input for the research topic
user_input = input("Please provide a topic: ")

# Define the task for the analyst agent
analyst_task = Task(
    description=f'Research on {user_input}',
    agent=analyser,
    expected_output="When the input is well researched, thoroughly analysed and recommendation is offered"
)

# Create the crew with the defined agents and task
investment_crew = Crew(
    agents=[researcher, analyser, recommend],
    tasks=[analyst_task],
    verbose=1,
    full_output=True,
)

# Execute the process

res = investment_crew.kickoff()

# Final Code

In [ ]:
from crewai import Agent, Task, Crew, Process
from composio_langchain import ComposioToolSet, Action, App
from langchain_google_genai import ChatGoogleGenerativeAI
import os
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

llm = ChatGoogleGenerativeAI(
        model="gemini-pro", verbose=True, temperature=0.9, google_api_key=os.getenv("GOOGLE_API_KEY")
    )

composio_toolset = ComposioToolSet()
tools = composio_toolset.get_tools(apps=[App.SERPAPI])

researcher = Agent(
  role='Investment Researcher',
  goal='Use SERP to research the top 2 results based on the input given to you and provide a report',
  backstory="""
  You are an expert Investment researcher. Using the information given to you, conduct comprehensive research using
  various sources and provide a detailed report. Dont pass in location as an argument to the tool
  """,
  verbose=True,
  allow_delegation=True,
  tools=tools,
  llm=llm
)


analyser = Agent(
  role='Investment Analyst',
  goal='Analyse the stock based on information available to it, use all the tools',
  backstory="""
  You are an expert Investment Analyst. You research on the given topic and analyse your research for insights.
  Note: Do not use SERP when youre writing the report""",
  verbose=True,
  tools=tools,
  llm=llm
)

recommend = Agent(
  role='Investment Recommendation',
  goal='Based on the analyst insights, you offer recommnedations',
  backstory="""
  You are an expert Investment Recommender. You understand the analyst insights and with your expertise suggest and offer
  advice on whether to invest or not. List the Pros and Cons as bullet points
  """,
  verbose=True,
  tools=tools,
  llm=llm
)
user_input = input(
        "Please provide a topic: "
    )


analyst_task = Task(
    description=f'Research on '+ user_input,
    agent = analyser,
    expected_output = "When the input is well researched, throughouly analysed and recommendation is offered"


)

    # Create the crew
investment_crew = Crew(
        agents=[researcher, analyser, recommend],
        tasks=[analyst_task],
        verbose=1,
        #process = Process.sequential,
        full_output = True,

    )

# Execution Flow
res = investment_crew.kickoff()